In [1]:
from timeit import default_timer as timer

start = timer()

In [ ]:
import os

os.environ["GROGUPY_ARCHITECTURE"] = "GPU"
import grogupy._environ
import numpy as np
import grogupy

print(grogupy._environ.GROGUPY_GET_ENVIRONMENT_VARIABLE("GROGUPY_PARALLEL_SIZE"))

import cupy as cp
from concurrent.futures import ThreadPoolExecutor

print("setup", (timer() - start), "s")

1


[Daniels-MacBook-Air.local:65027] shmem: mmap: an error occurred while determining whether or not /var/folders/yh/dx7xl94n3g52ts3td8qcxjcc0000gn/T//ompi.Daniels-MacBook-Air.501/jf.0/2833645568/sm_segment.Daniels-MacBook-Air.501.a8e60000.0 could be created.


ModuleNotFoundError: No module named 'cupy'

In [6]:
system_label = "/home/c_magpd/c_magex/dani/grogu_testing/Quick_redo2/CrIWSe.fdf"
infile = f"{system_label}"

kdirs = "xy"
kset = 3
eset = 100
esetp = 1000
emin = -30


outfile = f"{system_label.split('/')[-1]}_{kdirs}_{kset}_{eset}_0_120"


scf_xcf_orientation = np.array([0, 0, 1])
x = np.array([1, 0, 0])
y = np.array([np.cos(2 * np.pi / 3), np.sin(2 * np.pi / 3), 0])
z = np.array([0, 0, 1])
orientations = np.array([x, y, z])
tag = "Cr"

In [7]:
hamiltonian = grogupy.Hamiltonian(
    infile,
    scf_xcf_orientation,
)
print("hamiltonian", (timer() - start) / 60, "min")

tags = []
for at in hamiltonian._dh.atoms:
    tags.append(at.tag)
tags = np.array(tags)
cr = np.where(tags == tag)[0]


kspace = grogupy.KSpace(
    kdirs,
    kset,
)
contour = grogupy.Contour(
    emin,
    eset,
    esetp,
    eigfile=infile,
)
simulation = grogupy.Builder(orientations)
simulation.add_kspace(kspace)
simulation.add_contour(contour)
simulation.add_hamiltonian(hamiltonian)
simulation.setup_from_range(4, [cr.tolist(), cr.tolist()])
print("setup", (timer() - start) / 60, "min")


simulation.finalize()
print("finalize", (timer() - start) / 60, "min")
print(simulation)

FileNotFoundError: [Errno 2] No such file or directory: '/home/c_magpd/c_magex/dani/grogu_testing/Quick_redo2/CrIWSe.fdf'

In [ ]:
num_cpu = 4
builder = simulation
_tqdm = grogupy._tqdm._tqdm

# iterate over the reference directions (quantization axes)
G_mag = []
G_pair_ij = []
G_pair_ji = []

for orient in builder.ref_xcf_orientations:
    G_mag.append([])
    G_pair_ij.append([])
    G_pair_ji.append([])

    # obtain rotated exchange field and Hamiltonian
    for mag_ent in builder.magnetic_entities:
        G_mag[-1].append(
            cp.zeros(
                (builder.contour.eset, mag_ent.SBS, mag_ent.SBS),
                dtype="complex128",
            )
        )

    for pair in builder.pairs:
        G_pair_ij[-1].append(
            cp.zeros((builder.contour.eset, pair.SBS1, pair.SBS2), dtype="complex128")
        )
        G_pair_ji[-1].append(
            cp.zeros((builder.contour.eset, pair.SBS2, pair.SBS1), dtype="complex128")
        )
G_mag = cp.array(G_mag)
G_pair_ij = cp.array(G_pair_ij)
G_pair_ji = cp.array(G_pair_ji)

SBI = cp.array([m.spin_box_indices for m in builder.magnetic_entities])
SBI1 = cp.array([p.SBI1 for p in builder.pairs])
SBI2 = cp.array([p.SBI2 for p in builder.pairs])
Ruc = cp.array([p.supercell_shift for p in builder.pairs])

sc_off = cp.array(builder._dh.sc_off)
eset = builder.contour.eset


rotated_H = [H.H for H in builder._rotated_hamiltonians]
rotated_S = [H.S for H in builder._rotated_hamiltonians]

all_kpoints = np.array_split(builder.kspace.kpoints, num_cpu)
all_kweights = np.array_split(builder.kspace.weights, num_cpu)
samples = cp.array(builder.contour.samples.reshape(eset, 1, 1))
print("setup", (timer() - start) / 60, "min")


def process_gpu(gpu_number):
    with cp.cuda.Device(gpu_number):
        local_kpoints = all_kpoints[gpu_number]
        local_kweights = all_kweights[gpu_number]
        local_kpoints = cp.array(local_kpoints)
        local_kweights = cp.array(local_kweights)
        local_SBI = cp.array(SBI)
        local_SBI1 = cp.array(SBI1)
        local_SBI2 = cp.array(SBI2)
        local_Ruc = cp.array(Ruc)

        local_sc_off = cp.array(sc_off)
        local_samples = cp.array(samples)

        local_G_mag = cp.zeros_like(G_mag)
        local_G_pair_ij = cp.zeros_like(G_pair_ij)
        local_G_pair_ji = cp.zeros_like(G_pair_ji)

        for i in _tqdm(range(len(local_kpoints))):
            # weight of k point in BZ integral
            wk = local_kweights[i]
            k = local_kpoints[i]

            # iterate over reference directions
            for j in range(len(rotated_H)):
                # calculate Hamiltonian and Overlap matrix in a given k point
                # this generates the list of phases
                phases = cp.exp(-1j * 2 * cp.pi * k @ local_sc_off.T)

                # phases applied to the hamiltonian
                HK = cp.einsum("abc,a->bc", cp.array(rotated_H[j]), phases)
                SK = cp.einsum("abc,a->bc", cp.array(rotated_S[j]), phases)

                for e in range(eset):
                    Gk = cp.linalg.inv(SK * local_samples[e] - HK)

                    # store the Greens function slice of the magnetic entities
                    for l, sbi in enumerate(local_SBI):
                        local_G_mag[j][l][e] = Gk[..., sbi, :][..., sbi] * wk

                    for l, dat in enumerate(zip(local_SBI1, local_SBI2, local_Ruc)):
                        sbi1, sbi2, ruc = dat
                        phase = cp.exp(1j * 2 * cp.pi * k @ ruc.T)

                        # store the Greens function slice of the magnetic entities
                        local_G_pair_ij[j][l][e] += (
                            Gk[..., sbi1, :][..., sbi2] * wk * phase
                        )
                        local_G_pair_ji[j][l][e] += (
                            Gk[..., sbi2, :][..., sbi1] * wk / phase
                        )

    return local_G_mag, local_G_pair_ij, local_G_pair_ji


# Replace the GPU loop with this:
with ThreadPoolExecutor(max_workers=num_cpu) as executor:
    futures = [executor.submit(process_gpu, gpu_id) for gpu_id in range(num_cpu)]
    results = [future.result() for future in futures]
print("threadpool", (timer() - start) / 60, "min")

# Combine results
G_mag = cp.asnumpy(G_mag)
G_pair_ji = cp.asnumpy(G_pair_ji)
G_pair_ij = cp.asnumpy(G_pair_ij)
for G_mag_local, G_pair_ij_local, G_pair_ji_local in results:
    G_mag += cp.asnumpy(G_mag_local)
    G_pair_ij += cp.asnumpy(G_pair_ij_local)
    G_pair_ji += cp.asnumpy(G_pair_ji_local)

print("summing up", (timer() - start) / 60, "min")

# sum reduce partial results of mpi nodes
for j in range(len(builder._rotated_hamiltonians)):
    for l in range(len(SBI)):
        mag_ent = builder.magnetic_entities[l]
        mag_ent.Gii[j] = G_mag[j][l]

        mag_ent.calculate_energies(builder.contour.weights)
        mag_ent.fit_anisotropy_tensor(builder.ref_xcf_orientations)

    for l in range(len(SBI1)):
        pair = builder.pairs[l]
        pair.Gij[j] = G_pair_ij[j][l]
        pair.Gji[j] = G_pair_ji[j][l]

        pair.calculate_energies(builder.contour.weights)
        pair.fit_exchange_tensor(builder.ref_xcf_orientations)

print("energies and magnetism", (timer() - start) / 60, "min")

In [ ]:
simulation.times.times

In [ ]:
print("total", (timer() - start) / 60, "min")
print(simulation.to_magnopy())

In [ ]:
"""
kdirs = "xy"
kset = 5
eset = 100
total 13.19603161625176 min

kdirs = "xy"
kset = 3
eset = 100
total  min

"""